# 📊 Analisi Strategica HR: Performance, Turnover e Demografia

## Guida Completa per Manager HR

**Creato il:** 18 Luglio 2025
**Destinatari:** Manager HR, Direttori del Personale, Leadership aziendale
**Livello:** Principiante - Non richiede conoscenze tecniche

---

### 🎯 **Cosa troverai in questa analisi completa:**

1.  **🔍 Setup e Caricamento Dati** - Preparazione automatica dell'ambiente e caricamento del file `hr_data.csv`.
2.  **👥 Analisi Demografica** - Chi sono i nostri dipendenti? (Età, Genere, Stato Civile)
3.  **🏢 Analisi Organizzativa** - Come è composta la nostra azienda? (Dipartimenti, Anzianità di Servizio)
4.  **💰 Analisi Salariale** - Le retribuzioni sono eque e competitive? (Distribuzione, confronto tra dipartimenti e genere).
5.  **📈 Analisi di Performance** - Come performano i nostri team e individui?
6.  **🚪 Analisi del Turnover e della Retention** - Chi lascia l'azienda e perché? Calcolo del tasso di turnover e analisi dei dipendenti terminati.
7.  **🤝 Analisi delle Fonti di Assunzione** - Quali canali portano i talenti migliori?
8.  **💡 Report Esecutivo e Raccomandazioni** - Riepilogo dei principali insight e azioni strategiche consigliate.

### 🚀 **Come leggere questo notebook:**

-   **Celle Grigie con Codice:** Contengono il codice Python che esegue le analisi. Esegui ogni cella premendo **Shift+Enter**.
-   **Testo e Spiegazioni:** Forniscono il contesto e l'interpretazione dei risultati in un linguaggio chiaro per l'HR.
-   **📈 Grafici e Visualizzazioni:** Mostrano i dati in modo intuitivo.
-   **🔍 Insight:** Evidenziano le scoperte più importanti e le implicazioni pratiche per guidare le tue decisioni.

## 1. 🔧 Preparazione dell'Ambiente e Caricamento Dati

**❗ IMPORTANTE:** Esegui le due celle seguenti per prima cosa. Il codice si occuperà di:
1. Installare automaticamente le librerie necessarie per l'analisi.
2. Caricare il file `hr_data.csv` (CHE DEVE STARE NELLA STESSA DIR), pulire i dati (es. correggere formati di data errati), e calcolare nuove informazioni utili come l'età e gli anni di servizio.

In [ ]:
# 🔧 CONTROLLO E INSTALLAZIONE AUTOMATICA LIBRERIE
import subprocess
import sys

def installa_libreria(nome_libreria):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", nome_libreria])
        print(f"✅ {nome_libreria} installata con successo!")
    except subprocess.CalledProcessError:
        print(f"❌ Errore nell'installazione di {nome_libreria}")

# Lista delle librerie necessarie
librerie_necessarie = {
    'pandas': 'pandas',
    'numpy': 'numpy', 
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn'
}

print("🔍 Controllo librerie necessarie...")
librerie_mancanti = []

for nome_import, nome_pip in librerie_necessarie.items():
    try:
        __import__(nome_import)
        print(f"✅ {nome_import} - OK")
    except ImportError:
        print(f"❌ {nome_import} - MANCANTE")
        librerie_mancanti.append(nome_pip)

if librerie_mancanti:
    print(f"\n🔧 Installazione di {len(librerie_mancanti)} librerie mancanti...")
    for libreria in librerie_mancanti:
        installa_libreria(libreria)
    print("\n🔄 Riavvio dell'importazione...")
else:
    print("\n🎉 Tutte le librerie sono già installate!")

# ===== IMPORTAZIONE DELLE LIBRERIE =====
print("\n📦 Importazione librerie per l'analisi dati...")

try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from datetime import datetime
    import warnings
    
    warnings.filterwarnings('ignore')
    
    # Configurazione grafici per un aspetto professionale
    sns.set_style("whitegrid")
    sns.set_palette("husl", 8)
    plt.rcParams['figure.figsize'] = (12, 6)
    plt.rcParams['font.size'] = 12
    plt.rcParams['axes.titlesize'] = 16
    plt.rcParams['axes.labelsize'] = 14
    
    print("✅ Tutte le librerie importate con successo!")
    print("🚀 Ambiente pronto per l'analisi HR!")
    
except ImportError as e:
    print(f"❌ Errore durante l'importazione: {e}")
    print("💡 Prova a riavviare il kernel e rieseguire questa cella.")

In [ ]:
def carica_e_pulisci_dati(file_path='hr_data.csv'):
    """Carica, pulisce e pre-processa i dati HR da un file CSV."""
    try:
        print(f"📁 Tentativo di caricamento del file '{file_path}'...")
        df = pd.read_csv(file_path)
        print(f"✅ Dati caricati con successo! Trovati {len(df):,} record.")
    except FileNotFoundError:
        print(f"❌ ERRORE: File '{file_path}' non trovato. Assicurati che il file sia nella stessa cartella del notebook.")
        return None
    except Exception as e:
        print(f"❌ Si è verificato un errore durante il caricamento del file: {e}")
        return None

    print("🧹 Inizio pulizia e arricchimento dati...")
    
    # Correzione delle date malformate (es. 06/17,1989 -> 06/17/1989)
    for col in ['DateOfBirth', 'HiringDate', 'TerminationDate']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '/')
            df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Calcolo Età e Anzianità di Servizio
    oggi = pd.Timestamp('2025-07-18')
    if 'DateOfBirth' in df.columns:
        df['Eta'] = ((oggi - df['DateOfBirth']).dt.days / 365.25).astype(int)
    if 'HiringDate' in df.columns:
        # Se la data di terminazione non è presente, usa 'oggi', altrimenti usa la data di terminazione
        data_fine_servizio = df['TerminationDate'].fillna(oggi)
        df['AnniServizio'] = ((data_fine_servizio - df['HiringDate']).dt.days / 365.25).round(1)
        df['AnnoAssunzione'] = df['HiringDate'].dt.year

    # Standardizzazione Genere
    if 'Gender' in df.columns:
        df['Gender'] = df['Gender'].str.strip().replace({'M': 'Male', 'F': 'Female'})
        
    # Pulizia spazi extra in colonne di testo
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.strip()

    print(f"✅ Pulizia completata! Dataset finale con {len(df):,} dipendenti.")
    
    return df

# Esecuzione della funzione
df = carica_e_pulisci_dati()

if df is not None:
    print("\n📋 **Anteprima dei Dati Pronti per l'Analisi:**")
    display(df[['EmployeeName', 'Department', 'Position', 'Eta', 'AnniServizio', 'Salary', 'PerformanceScore', 'EmploymentStatus']].head())
    
    print("\n📊 **Riepilogo Dataset:**")
    dipendenti_attivi = df[df['EmploymentStatus'] == 'Active'].shape[0]
    salario_medio = df['Salary'].mean()
    eta_media = df['Eta'].mean()
    
    print(f"- Dipendenti totali: {len(df):,}")
    print(f"- Dipendenti attivi: {dipendenti_attivi:,}")
    print(f"- Salario medio: €{salario_medio:,.2f}")
    print(f"- Età media: {eta_media:.1f} anni")
else:
    print("\n🔴 Impossibile procedere senza dati. Controlla il file e riesegui.")

## 2. 👥 Analisi Demografica: Chi Sono i Nostri Dipendenti?

In questa sezione, esploriamo le caratteristiche demografiche della nostra forza lavoro per capire meglio chi sono le persone che compongono la nostra azienda. Analizzeremo la distribuzione per genere, l'età e lo stato civile.

In [ ]:
if df is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    # Grafico a Torta per la distribuzione di genere
    gender_counts = df['Gender'].value_counts()
    axes[0].pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
    axes[0].set_title('Distribuzione per Genere')
    
    # Grafico a Barre per il conteggio assoluto
    sns.countplot(ax=axes[1], x='Gender', data=df, palette='pastel')
    axes[1].set_title('Conteggio Dipendenti per Genere')
    axes[1].set_xlabel('Genere')
    axes[1].set_ylabel('Numero di Dipendenti')
    
    plt.tight_layout()
    plt.show()
    
    # 📊 INSIGHT DINAMICI GENERE
    female_pct = gender_counts.get('Female', 0) / len(df) * 100
    male_pct = gender_counts.get('Male', 0) / len(df) * 100
    female_count = gender_counts.get('Female', 0)
    male_count = gender_counts.get('Male', 0)
    
    print("🔍 **Insight Dinamici - Distribuzione per Genere:**")
    print(f"👥 **Composizione Forza Lavoro** (Totale: {len(df):,} dipendenti)")
    print(f"   • 👩 Donne: {female_count:,} dipendenti ({female_pct:.1f}%)")
    print(f"   • 👨 Uomini: {male_count:,} dipendenti ({male_pct:.1f}%)")
    
    # Analisi automatica dell'equilibrio
    print(f"\n💡 **Analisi Equilibrio di Genere:**")
    diff = abs(female_pct - male_pct)
    if diff <= 5:
        print(f"✅ **Distribuzione molto equilibrata**: Differenza di solo {diff:.1f} punti percentuali")
        print(f"   → Ottimo equilibrio per le politiche di diversità e inclusione")
    elif diff <= 15:
        print(f"📊 **Distribuzione accettabile**: Differenza di {diff:.1f} punti percentuali")
        majority_gender = 'donne' if female_pct > male_pct else 'uomini'
        print(f"   → Leggera prevalenza di {majority_gender}, monitorare nelle nuove assunzioni")
    else:
        print(f"⚠️  **Squilibrio significativo**: Differenza di {diff:.1f} punti percentuali")
        majority_gender = 'donne' if female_pct > male_pct else 'uomini'
        minority_gender = 'uomini' if female_pct > male_pct else 'donne'
        print(f"   → Forte prevalenza di {majority_gender}, considerare azioni per attrarre più {minority_gender}")
    
    # Raccomandazioni specifiche
    print(f"\n🎯 **Raccomandazioni HR:**")
    if female_pct < 30:
        print(f"📈 **Priorità**: Aumentare la rappresentanza femminile (attualmente {female_pct:.1f}%)")
        print(f"   → Rivedere job posting, canali di recruiting e politiche family-friendly")
    elif female_pct > 70:
        print(f"📈 **Priorità**: Aumentare la rappresentanza maschile (attualmente {male_pct:.1f}%)")
        print(f"   → Espandere canali di recruiting per attrarre candidati maschi")
    else:
        print(f"✅ **Mantenere l'equilibrio attuale** attraverso recruiting consapevole")
        print(f"   → Continuare a monitorare la distribuzione nelle nuove assunzioni")

In [ ]:
if df is not None:
    plt.figure(figsize=(14, 7))
    sns.histplot(df['Eta'], bins=20, kde=True, color='skyblue')
    plt.title('Distribuzione dell\'Età dei Dipendenti')
    plt.xlabel('Età')
    plt.ylabel('Numero di Dipendenti')
    
    eta_media = df['Eta'].mean()
    eta_mediana = df['Eta'].median()
    plt.axvline(eta_media, color='red', linestyle='--', label=f'Età Media: {eta_media:.1f}')
    plt.axvline(eta_mediana, color='green', linestyle='-', label=f'Età Mediana: {eta_mediana:.1f}')
    plt.legend()
    plt.show()
    
    # 📊 INSIGHT DINAMICI ETÀ
    eta_min = df['Eta'].min()
    eta_max = df['Eta'].max()
    
    # Analisi per fasce d'età
    fasce_eta = {
        'Giovani (< 30)': len(df[df['Eta'] < 30]),
        'Adulti (30-45)': len(df[(df['Eta'] >= 30) & (df['Eta'] <= 45)]),
        'Senior (46-55)': len(df[(df['Eta'] >= 46) & (df['Eta'] <= 55)]),
        'Esperti (> 55)': len(df[df['Eta'] > 55])
    }
    
    print("🔍 **Insight Dinamici - Distribuzione per Età:**")
    print(f"📊 **Statistiche Età** (Totale: {len(df):,} dipendenti)")
    print(f"   • 📈 Età media: {eta_media:.1f} anni")
    print(f"   • 📊 Età mediana: {eta_mediana:.1f} anni")
    print(f"   • 📏 Range: {eta_min}-{eta_max} anni")
    
    print(f"\n👥 **Distribuzione per Fasce d'Età:**")
    for fascia, count in fasce_eta.items():
        pct = count / len(df) * 100
        print(f"   • {fascia}: {count:,} dipendenti ({pct:.1f}%)")
    
    # Analisi automatica della composizione
    fascia_dominante = max(fasce_eta.items(), key=lambda x: x[1])
    pct_dominante = fascia_dominante[1] / len(df) * 100
    
    print(f"\n💡 **Analisi Composizione Anagrafica:**")
    print(f"• **Fascia dominante**: {fascia_dominante[0]} con {fascia_dominante[1]:,} dipendenti ({pct_dominante:.1f}%)")
    
    # Raccomandazioni basate sui dati
    print(f"\n🎯 **Raccomandazioni HR:**")
    
    # Analisi pensionamenti
    prossimi_pensionamenti = fasce_eta['Esperti (> 55)']
    if prossimi_pensionamenti > len(df) * 0.15:  # Più del 15%
        print(f"⚠️  **Rischio pensionamenti**: {prossimi_pensionamenti} dipendenti over 55 ({prossimi_pensionamenti/len(df)*100:.1f}%)")
        print(f"   → Priorità assoluta: pianificazione successione e knowledge transfer")
    elif prossimi_pensionamenti > 0:
        print(f"📋 **Monitorare pensionamenti**: {prossimi_pensionamenti} dipendenti over 55")
        print(f"   → Pianificare gradualmente la successione")
    
    # Analisi giovani talenti
    giovani = fasce_eta['Giovani (< 30)']
    if giovani < len(df) * 0.15:  # Meno del 15%
        print(f"📈 **Carenza giovani talenti**: Solo {giovani} dipendenti under 30 ({giovani/len(df)*100:.1f}%)")
        print(f"   → Intensificare recruiting junior e programmi graduate")
    else:
        print(f"✅ **Buon mix generazionale**: {giovani} giovani dipendenti ({giovani/len(df)*100:.1f}%)")
    
    # Analisi benefit
    if eta_media < 35:
        print(f"💡 **Focus benefit**: Forza lavoro giovane → Flessibilità, sviluppo carriera, work-life balance")
    elif eta_media > 45:
        print(f"💡 **Focus benefit**: Forza lavoro matura → Salute, pensione integrativa, flessibilità oraria")
    else:
        print(f"💡 **Focus benefit**: Mix equilibrato → Benefit diversificati per tutte le età")

## 3. 🏢 Analisi Organizzativa: Com'è Strutturata l'Azienda?

Questa sezione esamina la distribuzione dei dipendenti all'interno dell'azienda, analizzando la composizione dei dipartimenti e l'anzianità di servizio, un indicatore chiave della retention.

In [ ]:
if df is not None:
    plt.figure(figsize=(14, 8))
    sns.countplot(y='Department', data=df, order = df['Department'].value_counts().index, palette='viridis')
    plt.title('Distribuzione dei Dipendenti per Dipartimento')
    plt.xlabel('Numero di Dipendenti')
    plt.ylabel('Dipartimento')
    plt.show()
    
    # 📊 INSIGHT DINAMICI BASATI SUI DATI REALI
    dept_counts = df['Department'].value_counts()
    dept_percentages = (dept_counts / len(df) * 100).round(1)
    total_employees = len(df)
    
    print("🔍 **Insight Dinamici:**")
    print(f"📈 **Distribuzione per Dipartimento** (Totale: {total_employees:,} dipendenti)")
    
    # Top 3 dipartimenti
    top_3 = dept_counts.head(3)
    dept_names = list(top_3.index)
    
    print(f"\n🥇 **Top 3 Dipartimenti:**")
    for i, (dept, count) in enumerate(top_3.items(), 1):
        percentage = dept_percentages[dept]
        print(f"   {i}. **{dept}**: {count:,} dipendenti ({percentage}%)")
    
    # Analisi automatica
    largest_dept = dept_names[0]
    largest_count = top_3.iloc[0]
    largest_pct = dept_percentages.iloc[0]
    
    print(f"\n💡 **Analisi Automatica:**")
    print(f"• Il dipartimento **{largest_dept}** è il più numeroso con {largest_count:,} dipendenti ({largest_pct}% del totale)")
    
    if len(dept_names) >= 2:
        second_dept = dept_names[1]
        second_pct = dept_percentages.iloc[1]
        print(f"• Seguito da **{second_dept}** ({second_pct}%)")
        
        if len(dept_names) >= 3:
            third_dept = dept_names[2]
            third_pct = dept_percentages.iloc[2]
            print(f"• E **{third_dept}** ({third_pct}%)")
    
    # Raccomandazioni dinamiche
    print(f"\n🎯 **Raccomandazioni HR:**")
    if largest_pct > 40:
        print(f"⚠️  **Concentrazione elevata**: {largest_dept} rappresenta oltre il 40% della forza lavoro")
        print(f"   → Considerare diversificazione delle competenze e riduzione del rischio operativo")
    elif largest_pct > 30:
        print(f"📊 **Distribuzione bilanciata**: {largest_dept} ha una presenza significativa ma non eccessiva")
    else:
        print(f"✅ **Distribuzione equilibrata**: Nessun dipartimento domina eccessivamente")
    
    # Analisi dipartimenti piccoli
    small_depts = dept_counts[dept_counts < (total_employees * 0.05)]  # Meno del 5%
    if len(small_depts) > 0:
        print(f"\n🔍 **Dipartimenti piccoli** (< 5% del totale):")
        for dept, count in small_depts.items():
            pct = dept_percentages[dept]
            print(f"   • {dept}: {count} dipendenti ({pct}%)")
        print(f"   → Valutare se necessitano di rinforzi o ristrutturazione")

In [ ]:
if df is not None:
    plt.figure(figsize=(14, 7))
    sns.histplot(df['AnniServizio'], bins=15, kde=True, color='coral')
    plt.title('Distribuzione dell\'Anzianità di Servizio')
    plt.xlabel('Anni di Servizio')
    plt.ylabel('Numero di Dipendenti')
    
    anzianita_media = df['AnniServizio'].mean()
    plt.axvline(anzianita_media, color='blue', linestyle='--', label=f'Anzianità Media: {anzianita_media:.1f} anni')
    plt.legend()
    plt.show()

    # 📊 INSIGHT DINAMICI ANZIANITÀ
    anzianita_min = df['AnniServizio'].min()
    anzianita_max = df['AnniServizio'].max()
    anzianita_mediana = df['AnniServizio'].median()
    
    # Analisi per fasce di anzianità
    fasce_anzianita = {
        'Nuovi (0-2 anni)': len(df[df['AnniServizio'] <= 2]),
        'Junior (3-5 anni)': len(df[(df['AnniServizio'] > 2) & (df['AnniServizio'] <= 5)]),
        'Esperti (6-10 anni)': len(df[(df['AnniServizio'] > 5) & (df['AnniServizio'] <= 10)]),
        'Senior (11-15 anni)': len(df[(df['AnniServizio'] > 10) & (df['AnniServizio'] <= 15)]),
        'Veterani (> 15 anni)': len(df[df['AnniServizio'] > 15])
    }
    
    print("🔍 **Insight Dinamici - Anzianità di Servizio:**")
    print(f"📊 **Statistiche Anzianità** (Totale: {len(df):,} dipendenti)")
    print(f"   • 📈 Anzianità media: {anzianita_media:.1f} anni")
    print(f"   • 📊 Anzianità mediana: {anzianita_mediana:.1f} anni")
    print(f"   • 📏 Range: {anzianita_min:.1f}-{anzianita_max:.1f} anni")
    
    print(f"\n👥 **Distribuzione per Fasce di Anzianità:**")
    for fascia, count in fasce_anzianita.items():
        pct = count / len(df) * 100
        print(f"   • {fascia}: {count:,} dipendenti ({pct:.1f}%)")
    
    # Analisi automatica della composizione
    fascia_dominante = max(fasce_anzianita.items(), key=lambda x: x[1])
    pct_dominante = fascia_dominante[1] / len(df) * 100
    
    print(f"\n💡 **Analisi Composizione Anzianità:**")
    print(f"• **Fascia dominante**: {fascia_dominante[0]} con {fascia_dominante[1]:,} dipendenti ({pct_dominante:.1f}%)")
    
    # Raccomandazioni basate sui dati
    print(f"\n🎯 **Raccomandazioni HR:**")
    
    # Analisi retention
    nuovi_dipendenti = fasce_anzianita['Nuovi (0-2 anni)']
    if nuovi_dipendenti > len(df) * 0.3:  # Più del 30%
        print(f"📈 **Crescita rapida**: {nuovi_dipendenti} dipendenti con meno di 2 anni ({nuovi_dipendenti/len(df)*100:.1f}%)")
        print(f"   → Focus su onboarding e programmi di retention per nuovi assunti")
    elif nuovi_dipendenti < len(df) * 0.15:  # Meno del 15%
        print(f"⚠️  **Poche nuove assunzioni**: Solo {nuovi_dipendenti} dipendenti under 2 anni ({nuovi_dipendenti/len(df)*100:.1f}%)")
        print(f"   → Valutare se intensificare il recruiting o se è una scelta strategica")
    else:
        print(f"✅ **Equilibrio sano**: {nuovi_dipendenti} nuovi dipendenti ({nuovi_dipendenti/len(df)*100:.1f}%)")
    
    # Analisi veterani
    veterani = fasce_anzianita['Veterani (> 15 anni)']
    if veterani > len(df) * 0.2:  # Più del 20%
        print(f"🏆 **Forte retention**: {veterani} veterani con oltre 15 anni ({veterani/len(df)*100:.1f}%)")
        print(f"   → Ottima capacità di trattenere i talenti, valorizzare la loro esperienza")
    elif veterani > 0:
        print(f"📋 **Nucleo esperto**: {veterani} veterani con oltre 15 anni")
        print(f"   → Utilizzare la loro esperienza per mentoring e knowledge transfer")
    
    # Analisi turnover potenziale
    if anzianita_media < 5:
        print(f"💡 **Focus retention**: Anzianità media bassa → Programmi di engagement e crescita")
    elif anzianita_media > 12:
        print(f"💡 **Focus innovazione**: Anzianità media alta → Bilanciare con nuovi talenti e idee")
    else:
        print(f"💡 **Equilibrio ottimale**: Mix sano di esperienza e nuove energie")

## 4. 💰 Analisi Salariale: Equità e Competitività

L'analisi delle retribuzioni è fondamentale per garantire equità interna e competitività sul mercato del lavoro. In questa sezione, esaminiamo come sono distribuiti i salari e come variano tra dipartimenti e genere.

In [ ]:
if df is not None:
    plt.figure(figsize=(15, 8))
    # Ordiniamo i dipartimenti per salario mediano per una migliore leggibilità
    order = df.groupby('Department')['Salary'].median().sort_values(ascending=False).index
    sns.boxplot(x='Department', y='Salary', data=df, order=order, palette='magma')
    plt.title('Distribuzione degli Stipendi per Dipartimento')
    plt.xlabel('Dipartimento')
    plt.ylabel('Salario Annuo Lordo (€)')
    plt.xticks(rotation=45, ha='right')
    plt.show()

    print("🔍 **Insight:**")
    print("- I dipartimenti 'IT/IS' e 'Engineering' mostrano i salari mediani più alti, indicando ruoli di alta specializzazione.")
    print("- Il dipartimento 'Production' ha una gamma salariale più ristretta e salari mediani più bassi, in linea con la natura dei ruoli.")
    print("- L'analisi degli outlier (i punti isolati) può aiutare a identificare situazioni salariali anomale o top performer con retribuzioni elevate.")

In [ ]:
if df is not None:
    plt.figure(figsize=(15, 8))
    sns.barplot(x='Department', y='Salary', hue='Gender', data=df, palette='coolwarm', ci=None)
    plt.title('Salario Medio per Dipartimento e Genere')
    plt.xlabel('Dipartimento')
    plt.ylabel('Salario Medio Annuo Lordo (€)')
    plt.xticks(rotation=45, ha='right')
    plt.show()

    print("🔍 **Insight (Pay Gap Analysis):**")
    print("- Questo grafico è uno strumento essenziale per l'analisi del pay gap. A colpo d'occhio, possiamo confrontare la retribuzione media di uomini e donne in ogni dipartimento.")
    print("- Nella maggior parte dei dipartimenti, le differenze salariali medie tra generi sembrano minime. Tuttavia, ogni discrepanza, anche piccola, merita un'indagine più approfondita per garantire che sia giustificata da fattori oggettivi (es. anzianità, ruolo, performance) e non da bias di genere.")

## 5. 📈 Analisi di Performance

Capire la performance dei dipendenti è vitale per la gestione dei talenti, la pianificazione della formazione e lo sviluppo organizzativo. Analizziamo come si distribuiscono i punteggi di performance all'interno dell'azienda.

In [ ]:
if df is not None:
    plt.figure(figsize=(12, 7))
    performance_order = ['Needs Improvement', 'Fully Meets', 'Exceeds']
    sns.countplot(x='PerformanceScore', data=df, order=performance_order, palette='rocket')
    plt.title('Distribuzione dei Punteggi di Performance')
    plt.xlabel('Punteggio di Performance')
    plt.ylabel('Numero di Dipendenti')
    plt.show()
    
    perf_counts = df['PerformanceScore'].value_counts(normalize=True) * 100
    print("🔍 **Insight:**")
    print(f"- La stragrande maggioranza dei dipendenti ({perf_counts.get('Fully Meets', 0):.1f}%) soddisfa pienamente le aspettative ('Fully Meets').")
    print(f"- Una quota significativa ({perf_counts.get('Exceeds', 0):.1f}%) supera le aspettative, rappresentando i nostri top performer.")
    print(f"- Una piccola percentuale ({perf_counts.get('Needs Improvement', 0):.1f}%) necessita di miglioramenti; questo è il gruppo su cui concentrare gli sforzi di coaching e sviluppo.")

## 6. 🚪 Analisi del Turnover e della Retention

Il turnover dei dipendenti è uno dei KPI più critici per l'HR, con un impatto diretto sui costi, sul morale e sulla produttività. In questa sezione, calcoliamo il tasso di turnover e analizziamo chi ha lasciato l'azienda e perché.

In [ ]:
if df is not None:
    # Calcolo del turnover
    dipendenti_terminati = df[df['EmploymentStatus'] != 'Active'].shape[0]
    dipendenti_totali = len(df)
    turnover_rate = (dipendenti_terminati / dipendenti_totali) * 100
    
    # Visualizzazione
    status_counts = df['EmploymentStatus'].value_counts()
    plt.figure(figsize=(10, 6))
    sns.barplot(x=status_counts.index, y=status_counts.values, palette='coolwarm')
    plt.title('Stato Occupazionale dei Dipendenti')
    plt.ylabel('Numero di Dipendenti')
    plt.xlabel('Stato')
    
    print(f"🔴 **Tasso di Turnover Complessivo: {turnover_rate:.1f}%**")
    plt.show()
    
    print("🔍 **Insight:**")
    print(f"- Su {dipendenti_totali} dipendenti totali nel dataset, {dipendenti_terminati} hanno lasciato l'azienda.")
    print("- La causa principale di cessazione è volontaria ('Voluntarily Terminated'). Questo è un segnale chiave: dobbiamo capire le ragioni che spingono i dipendenti a cercare altre opportunità.")

In [ ]:
if df is not None and 'Voluntarily Terminated' in df['EmploymentStatus'].unique():
    df_terminated = df[df['EmploymentStatus'] == 'Voluntarily Terminated']
    
    if not df_terminated.empty:
        plt.figure(figsize=(14, 7))
        sns.countplot(y='Department', data=df_terminated, order=df_terminated['Department'].value_counts().index, palette='viridis_r')
        plt.title('Dimissioni Volontarie per Dipartimento')
        plt.xlabel('Numero di Dipendenti che si sono dimessi')
        plt.ylabel('Dipartimento')
        plt.show()
        
        print("🔍 **Insight:**")
        print("- L'analisi delle dimissioni volontarie per dipartimento può rivelare problemi specifici legati a cultura, management o carico di lavoro in determinate aree dell'azienda.")
        print("- Se un dipartimento mostra un numero sproporzionato di dimissioni, è un campanello d'allarme che richiede un'indagine immediata, ad esempio tramite focus group o exit interview più approfondite.")
    else:
        print("✅ Nessun dipendente si è dimesso volontariamente nel dataset.")

## 7. 🤝 Analisi delle Fonti di Assunzione

Da dove provengono i nostri migliori talenti? Analizzare l'efficacia delle fonti di assunzione ci permette di ottimizzare il budget di recruiting e concentrare gli sforzi sui canali che garantiscono le performance migliori.

In [ ]:
if df is not None:
    performance_map = {'Needs Improvement': 1, 'Fully Meets': 2, 'Exceeds': 3}
    df['PerformanceValue'] = df['PerformanceScore'].map(performance_map)

    avg_perf_by_source = df.groupby('RecruitmentSource')['PerformanceValue'].mean().sort_values(ascending=False)

    plt.figure(figsize=(14, 8))
    sns.barplot(x=avg_perf_by_source.values, y=avg_perf_by_source.index, palette='cubehelix')
    plt.title('Efficacia delle Fonti di Assunzione (Performance Media)')
    plt.xlabel('Performance Media (1=Needs Improvement, 3=Exceeds)')
    plt.ylabel('Fonte di Assunzione')
    plt.xlim(1, 3) # Imposta il limite dell'asse x per rappresentare la scala di performance
    plt.axvline(df['PerformanceValue'].mean(), color='red', linestyle='--', label='Performance Media Aziendale')
    plt.legend()
    plt.show()

    print("🔍 **Insight:**")
    print("- Questo grafico mostra quali canali di recruiting portano dipendenti con la performance media più alta.")
    print("- Le fonti che si posizionano al di sopra della linea tratteggiata (media aziendale) sono quelle più efficaci. Potremmo considerare di investire maggiormente in canali come 'Employee Referral' e 'LinkedIn', che storicamente portano talenti con performance elevate.")
    print("- Le fonti con performance medie inferiori potrebbero richiedere una revisione della strategia di sourcing o dei messaggi utilizzati.")

## 8. 💡 Report Esecutivo e Raccomandazioni Strategiche

Di seguito un riepilogo dei principali insight emersi dall'analisi e le azioni strategiche consigliate per la leadership e il team HR.

### Riepilogo dei Key Findings:

1.  **Forza Lavoro Stabile e Matura:** L'azienda ha una forza lavoro con un'età media consolidata e un'anzianità di servizio che indica una buona capacità di retention di base. La distribuzione di genere è abbastanza bilanciata.

2.  **Performance Solida:** La grande maggioranza dei dipendenti soddisfa o supera le aspettative. Esiste un solido nucleo di top performer ('Exceeds') e un gruppo ristretto ma identificato che necessita di supporto ('Needs Improvement').

3.  **Turnover Volontario come Sfida Principale:** La causa predominante di uscita dall'azienda è volontaria. Questo è il segnale più critico: i dipendenti scelgono di andarsene. L'analisi per dipartimento può aiutare a localizzare i punti critici.

4.  **Differenze Salariali da Monitorare:** Sebbene non emergano enormi disparità di genere a livello macro, è fondamentale un monitoraggio costante e approfondito del pay gap a parità di ruolo e anzianità per garantire equità assoluta.

5.  **Efficacia del Recruiting:** I canali come le segnalazioni dei dipendenti (Employee Referral) e LinkedIn sembrano portare i talenti con le performance più elevate, suggerendo un ottimo ritorno sull'investimento.

### Raccomandazioni Strategiche:

| Area di Intervento | Raccomandazione Specifica |
| :--- | :--- |
| **Retention e Turnover** | **1. Lanciare un Programma di 'Stay Interview':** Invece di aspettare le exit interview, condurre colloqui proattivi con i dipendenti chiave (specialmente nei dipartimenti con più alto turnover) per capire cosa li motiva e cosa potrebbe spingerli a restare. <br> **2. Analizzare i Dati delle Exit Interview:** Rivedere sistematicamente i dati delle interviste di uscita per identificare pattern ricorrenti (es. problemi con il management, mancanza di crescita, carico di lavoro) e presentare un report trimestrale alla leadership. |
| **Sviluppo e Performance** | **1. Creare Piani di Sviluppo Personalizzati:** Utilizzare i dati di performance per creare percorsi mirati. Coaching e mentoring per il gruppo 'Needs Improvement'; opportunità di leadership e progetti sfidanti per i 'Top Performer'. <br> **2. Riconoscere i Top Performer:** Implementare un sistema di riconoscimento (non necessariamente monetario) per i dipendenti che superano le aspettative, per aumentarne l'engagement e la visibilità. |
| **Equità e Inclusione** | **1. Condurre un'Analisi Approfondita sul Pay Gap:** Andare oltre la media di dipartimento e analizzare il pay gap a parità di ruolo, livello e anni di esperienza. Stabilire un piano d'azione per correggere eventuali disallineamenti ingiustificati. <br> **2. Promuovere la Leadership Femminile:** Valutare la rappresentanza di genere nei ruoli manageriali e di leadership e, se necessario, creare programmi di mentoring per accelerare la crescita delle donne in azienda. |
| **Talent Acquisition** | **1. Potenziare il Programma di Referral:** Visti i risultati eccellenti, rafforzare il programma di segnalazione dei dipendenti con incentivi migliori e una campagna di comunicazione interna per aumentarne la partecipazione. <br> **2. Ottimizzare gli Investimenti:** Allocare una porzione maggiore del budget di recruiting verso i canali che dimostrano di portare talenti con performance più elevate, come LinkedIn. |